In [ ]:
%run base.ipynb

# EPIC 1 - User Story DF-24
## Erstellung eines Notebooks und aufrufen der Daten per Dataland-API

Als QA-Lab-Team möchten wir ein Notebook erstellen, mit dem wir eine Anfrage an die Dataland-API senden, sodass wir anhand der Berichts-ID den Unternehmensbericht erhalten.

Beschreibung:<br>
Nutzen von Dataland Endpoint /data/nuclear-and-gas/companies/{companyId} im Nuclear & Gas Controller. Die Funktionalität wird außerhalb des Notebooks implementiert.

Akzeptanzkriterien:<br>
    - Notebook ist auf jedem Laptop ausführbar<br>
    - Anfrage an Dataland gibt Daten zurück

## 1.Schritt: Dataset mit Hilfe der Company_ID über die Dataland_API aufrufen
Mithilfe der Company_ID wird die Dataland_API aufgerufen. Nachdem Eintragen aus welcher Periode das Dataset sein soll, ist es möglich den Wert 1 zu erlangen.

In [ ]:
import dataland_qa_lab.dataland.get_data as qa
from dataland_qa_lab.utils import config

dataland_client = config.get_config().dataland_client

company_id = "4423c691-0436-423f-abcb-0a08127ee848"
year = "2024"

# Laden aller Datasets aus jeder Periode
all_datasets = qa.get_all_company_datasets(company_id=company_id)

# Data_ID der gewünschten Periode des Datensets erhalten
data_id = qa.get_data_id_by_year(company_id=company_id, year=year)
print("Data_ID = " + data_id)

# Laden eines bestimmten Datensets einer bestimmten Periode
dataset_by_year = qa.get_dataset_by_year(company_id=company_id, year=year)

# Rückgabe des ersten Wertes in dem bestimmten Datenset
section_426 = qa.get_value1_by_year(company_id=company_id, year=year)
print("Section426 = " + section_426)

# Erhalten der Datenreferenz in Bytes, wenn eine Datenquelle hinterlegt ist
datasource_reference_bytes = qa.get_datasource_reference_bytes(company_id=company_id, year=year)

## Daten aus dem Dokument extrahieren und in Text umwandeln - DEMO

In [ ]:
import io

import pypdf  # type: ignore

dataset_section426 = dataset_by_year.data.general.general.nuclear_energy_related_activities_section426
page_number = int(dataset_section426.data_source.page)

file_id = dataset_section426.data_source.file_reference
file_name = dataset_section426.data_source.file_name
tag_name = dataset_section426.data_source.tag_name
page = dataset_section426.data_source.page

pdf = dataland_client.documents_api.get_document(file_id)
pdf_stream = io.BytesIO(pdf)
pdf_reader = pypdf.PdfReader(pdf_stream)

In [ ]:
import dataland_qa_lab.dataland.data_extraction as qa_lab

relevant_page = qa_lab.get_relevant_page_of_pdf(page_number, pdf_reader)

text_of_page = qa_lab.extraxt_text_of_pdf(relevant_page)

section_426 = qa_lab.extract_section_426(text_of_page)

print(section_426)